<a href="https://colab.research.google.com/github/gugi200/final_project/blob/main/scikit_models_final_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install wandb -qU
!wandb login 3014974e724f01c4d63f956fa13fd7f0463e16d4
!pip install torchmetrics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 252.8/252.8 kB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.0 MB/s eta 0:00:00
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 5.6 MB/s eta 0:00:00


In [ ]:
# processing
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# ML libraries
import sklearn
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC


from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.preprocessing import PolynomialFeatures
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler


from sklearn.decomposition import PCA

from sklearn.pipeline import make_pipeline

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# torch functionality functions
import torch

# for downloading the data
from pathlib import Path
from torchvision.datasets import FashionMNIST
from torchvision import transforms
from torchvision import datasets
from sklearn.utils import Bunch

# helper functions
import wandb
from timeit import default_timer as timer
from tqdm.auto import tqdm

# download data for testing

In [ ]:
# download the test data

from torchvision.datasets import FashionMNIST
from torchvision import transforms
train_data = FashionMNIST(
    root='~/.pytorch/F_MNIST_data',
    download=True,
    train=True,
    target_transform=None)

test_data = FashionMNIST(
    root='~/.pytorch/F_MNIST_data',
    download=True,
    train=False,
    target_transform=None)

class_names = [x.replace('/', '_').replace('-', '') for x in train_data.classes]

transform = transforms.ToPILImage()



from pathlib import Path

data_path = Path("data/")
image_path1 = data_path / "FashionMNIST" / "train"

if image_path1.is_dir():
    print('directory already exists')
else:
    print('creating folder')
    for class_name in class_names:
        image_class_path = image_path1 / class_name
        image_class_path.mkdir(parents=True, exist_ok=True)

    for idx, (train_img, target) in enumerate(zip(train_data.data[:3000], train_data.targets[:3000])):
        train_data_PIL = transform(train_img)
        file = "data_" + str(idx) + '.jpg'
        # print(target)
        # print(data_path / "FashionMNIST"/"train" / class_names[target] /file)
        train_data_PIL.save(data_path / "FashionMNIST"/"train" / class_names[target] /file)


data_path = Path("data/")
image_path2 = data_path / "FashionMNIST" / "test"
if image_path2.is_dir():
    print('directory already exists')
else:
    print('creating folder')
    for class_name in class_names:
        image_class_path = image_path2 / class_name
        image_class_path.mkdir(parents=True, exist_ok=True)
        image_class_path.mkdir(parents=True, exist_ok=True)

    for idx, (test_img, target) in enumerate(zip(test_data.data[:250], test_data.targets[:250])):
        test_data_PIL = transform(test_img)
        file = "data_" + str(idx) + '.jpg'
        test_data_PIL.save(data_path / "FashionMNIST"/"test" / class_names[target] /file)





dataset = datasets.ImageFolder(root=Path("data/FashionMNIST/train"),
                                  transform=np.array,
                                  target_transform=None) # transform for label



class_names = dataset.classes

def torch_dataset_to_numpy(dataset, target_names):
    for i, (data, target) in enumerate(dataset):

        if i==0:
            data_numpy = data
            data_numpy = np.expand_dims(data_numpy, axis=0)
            target_numpy = np.array([target])
            target_numpy = np.expand_dims(target_numpy, axis=0)
        else:
            data = np.expand_dims(data, axis=0)
            target = np.expand_dims(np.array([target]), axis=0)
            data_numpy = np.append(data_numpy, data, axis=0)
            target_numpy = np.append(target_numpy, target, axis=0)

    return Bunch(data=data_numpy, target=target_numpy, target_names=target_names)


custom_dataset = torch_dataset_to_numpy(dataset, class_names)



100%|██████████| 26421880/26421880 [00:01<00:00, 16360258.47it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



100%|██████████| 29515/29515 [00:00<00:00, 306469.25it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



100%|██████████| 4422102/4422102 [00:00<00:00, 5540179.51it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw



100%|██████████| 5148/5148 [00:00<00:00, 16041810.54it/s]


Extracting /root/.pytorch/F_MNIST_data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to /root/.pytorch/F_MNIST_data/FashionMNIST/raw

creating folder
creating folder


In [ ]:
from sklearn.datasets import load_iris
iris = load_iris()
iris.data.shape, iris.target.shape

((150, 4), (150,))

# Data preprocessing

In [ ]:
# split the data
X_train, X_test, y_train, y_test = train_test_split(custom_dataset.data,
                                                    custom_dataset.target,
                                                    test_size=0.25,
                                                    random_state=42,
                                                    shuffle=True)


X_train, X_test = X_train[:, :, :, 0].reshape((2250, 784)), X_test[:, :, :, 0].reshape((750, 784))




X_train.shape, y_train.shape






((2250, 784), (2250, 1))

# create pipelines

In [ ]:
param_RF = {
'max_depth': [2, 3, 4, 5, 6, 7, 8],
'n_estimators': [50, 75, 100, 125, 150]
}

grid_rf = GridSearchCV(RandomForestClassifier(random_state=42), param_RF, cv=5, n_jobs=-1)
grid_rf.fit(X_train, y_train)
print('random forest')
# print('results: ', grid_rf.cv_results_)
print('Best estimator: ', grid_rf.best_estimator_)
print('Best params: ', grid_rf.best_params_)
print('Best score: ', grid_rf.best_score_)



param_DT = {
'max_depth': [2, 3, 4, 5, 6, 7, 8]
}

grid_DT = GridSearchCV(DecisionTreeClassifier(random_state=42), param_DT, cv=5, n_jobs=-1)
grid_DT.fit(X_train, y_train)
print('decisionb forest')
# print('results: ', grid_DT.cv_results_)
print('Best estimator: ', grid_DT.best_estimator_)
print('Best params: ', grid_DT.best_params_)
print('Best score: ', grid_DT.best_score_)




param_svc = {
'C': [2, 3, 4, 5, 6, 7, 8],
'gamma': [50, 75, 100, 125, 150]
}

grid_svc_std = GridSearchCV(SVC(probability=True), param_svc, cv=5, n_jobs=-1)
stdScaled = StandardScaler()
stdScaled = stdScaled.fit(X_train)
scaled_X_train = stdScaled.transform(X_train)
scaled_X_test = stdScaled.fit(X_test)
grid_svc_std.fit(scaled_X_train, y_train)
print('svc standard scaler')
# print('results: ', grid_svc_std.cv_results_)
print('Best estimator: ', grid_svc_std.best_estimator_)
print('Best params: ', grid_svc_std.best_params_)
print('Best score: ', grid_svc_std.best_score_)



param_grid_svc = {
    'gamma': [0.001, 0.01, 0.1, 1, 10, 100],
    'C': [0.001, 0.01, 0.1, 1, 10, 100]
    }


grid_svc_mm = GridSearchCV(SVC(probability=True), param_grid_svc, cv=5, n_jobs=-1)
mmScaled = MinMaxScaler()
mmScaled = mmScaled.fit(X_train)
scaled_X_train = mmScaled.transform(X_train)
grid_svc_mm.fit(scaled_X_train, y_train)
print('svc minmax')
# print('results: ', grid_svc_mm.cv_results_)
print('Best estimator: ', grid_svc_mm.best_estimator_)
print('Best params: ', grid_svc_mm.best_params_)
print('Best score: ', grid_svc_mm.best_score_)











/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:909: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)


random forest
Best estimator:  RandomForestClassifier(max_depth=8, random_state=42)
Best params:  {'max_depth': 8, 'n_estimators': 100}
Best score:  0.8044444444444444
decisionb forest
Best estimator:  DecisionTreeClassifier(max_depth=8, random_state=42)
Best params:  {'max_depth': 8}
Best score:  0.7022222222222222


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


svc standard scaler
Best estimator:  SVC(C=2, gamma=50, probability=True)
Best params:  {'C': 2, 'gamma': 50}
Best score:  0.10800000000000001


/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


svc minmax
Best estimator:  SVC(C=10, gamma=0.01, probability=True)
Best params:  {'C': 10, 'gamma': 0.01}
Best score:  0.8368888888888888


In [ ]:
def plot_precision_recal_curve(model, y_test, X_test, title):
    # precision, recall, threshold = precision_recall_curve(y_test, model.predict_proba(X_test))
    # plt.figure()
    # plt.plot(precision, recall, title='precision-recall curve ' + title)
    # plt.xlabel('precision')
    # plt.ylabel('recall')
    # plt.legend(loc='best')


    false_posR, true_posR = th = roc_curve(y_test, model.predict_proba(X_test))
    plt.figure()
    plt.plot(false_posR, true_posR)
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.legend(loc='best')

plot_precision_recal_curve(grid_rf, y_test, X_test, 'Random Forest')
plot_precision_recal_curve(grid_DT, y_test, X_test, 'Decision Tree')
 #plot_precision_recal_curve(grid_svc_mm, y_test, X_test, 'SVC MinMax')
# plot_precision_recal_curve(grid_svc_std, y_test. X_test, 'SVC Standard')


